In [4]:
from enhance_ocod.labelling.ner_spans import lfs
from enhance_ocod.labelling.weak_labelling import (process_dataframe_batch, create_flat_tag, 
create_commercial_park_tag, remove_overlapping_spans, get_overlap_stats)
import pandas as pd 




In [7]:
df = pd.read_csv('data/enhanced_ocod_data_and_gt/ground_truth_dev_set_labels.csv')
#df = pd.read_csv("data/OCOD_FULL_2022_02.csv").rename(columns = {'Property Address':'property_address'})
df = df['property_address'].drop_duplicates().reset_index(drop=True).to_frame()


df= create_flat_tag(df)

df = create_commercial_park_tag(df)

/home/jonno/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:263: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['flat_tag'] = df[text_column].str.contains(
/home/jonno/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:271: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['commercial_park_tag'] = df[text_column].str.contains(


In [12]:
test = process_dataframe_batch(df, 
                           batch_size = 1000,
                           text_column = 'property_address',
                           include_function_name = False,
                           save_intermediate = False,
                           output_dir = None,
                           verbose = True)

remove_overlapping_spans(test)

Processing batches: 100%|██████████| 2/2 [00:01<00:00,  1.91it/s]


Processing complete:
  - Processed 1998 rows successfully
  - Found 10094 total spans
  - Average 5.05 spans per row


In [10]:
get_overlap_stats(test)

{'total_rows': 1998,
 'rows_with_overlaps': 1048,
 'overlap_percentage': 52.45245245245245,
 'total_spans_before': 10094,
 'total_overlaps_removed': 1623,
 'spans_after': 8471}

In [13]:
test[0:3]

[{'text': '207 sloane street, london (sw1x 9qx)',
  'spans': [{'text': '207', 'start': 0, 'end': 3, 'label': 'STREET_NUMBER'},
   {'text': 'sloane street', 'start': 4, 'end': 17, 'label': 'STREET'},
   {'text': 'london', 'start': 19, 'end': 25, 'label': 'CITY'},
   {'text': 'sw1x 9qx', 'start': 27, 'end': 35, 'label': 'POSTCODE'}],
  'row_id': 0},
 {'text': 'apartment 533, block 11 spectrum, blackfriars road, salford (m3 7ee)',
  'spans': [{'text': 'apartment', 'start': 0, 'end': 9, 'label': 'UNIT_TYPE'},
   {'text': '533', 'start': 10, 'end': 13, 'label': 'UNIT_ID'},
   {'text': 'blackfriars road', 'start': 34, 'end': 50, 'label': 'STREET'},
   {'text': 'salford', 'start': 52, 'end': 59, 'label': 'CITY'},
   {'text': 'm3 7ee', 'start': 61, 'end': 67, 'label': 'POSTCODE'}],
  'row_id': 1},
 {'text': 'flat 9, st. james heights, paradise walk, bexhill-on-sea (tn40 2lg)',
  'spans': [{'text': 'flat', 'start': 0, 'end': 4, 'label': 'UNIT_TYPE'},
   {'text': '9', 'start': 5, 'end': 6, 'labe

In [14]:
from enhance_ocod.training import convert_weak_labels_to_standard_format, evaluate_weak_labels



In [ ]:
test2 = convert_weak_labels_to_standard_format(test)

evaluate_weak_labels(test2, )